# Байесовский подход к оценке А/Б-тестов: сравнение выручки  

**Содержание**
- Введение
-- Пример теста
-- Пример данных
-- Разные уровни моделей и сравнения
- Генерация данных покупок
- Моделирование и сравнение средних выручек на пользователя
- Моделирование и сравение распределений выручки на пользователя
- Моделирование и сравнение покупок пользователей
- Заключение
- Благодарности
- Ссылки

### Введение

Кроме конверсий нужно оценивать выручку.
Или величины вроде длительности просмотра.

"Конверсиями зарплату не заплатишь".

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(7)

import scipy.stats as stats
import plotly.graph_objects as go

Ситуация примерно такая:
пришло N пользователей, каждый совершил k покупок, у каждой покупки своя стоимость.
Например:

In [ ]:
pd.DataFrame([
    {'experiment_group': 'A', 'user_id': 1, 'timestamp': '01.01.2021 10:00:00', 'purchase_value_usd': 5},
    {'experiment_group': 'B', 'user_id': 2, 'timestamp': '01.01.2021 10:05:00', 'purchase_value_usd': 3},
    {'experiment_group': 'A', 'user_id': 1, 'timestamp': '01.01.2021 10:06:00', 'purchase_value_usd': 10},
    {'experiment_group': 'A', 'user_id': 3, 'timestamp': '01.01.2021 10:07:00', 'purchase_value_usd': 5},
])

Как правило интересует, в какой группе будет выше прибыль относительно потраченных на маркетинг денег.

Если трафик в группах более-менее одинаковый, то можно сравнивать среднюю выручку на пользователя.  
Иногда - выручку на сессию.

Общий подход примерно такой же, как в случае конверсий.  
Выбирается модель.  
Вычисляется распределение параметров.  
Делается оценка и сравнение интересующих метрик в группах.

В качестве основной метрики будет использоваться выручка на пользователя.

Можно строить модель на разном уровне детализации.

Попытаться промоделировать процесс покупок.
Т.е. для каждого пользователя делать предсказание отдельных покупок.
Можно смотреть время между покупками.
Можно ограничиться количеством и суммой каждой из покупок.

Можно не детализировать модель до уровня покупок.
Строить распределение суммарной выручки на пользоователя (LTV).

Наконец, можно построить модель для оценки средних значений чеков.

Последний вариант - модель для оценки средней выручки на пользователя - кажется требующей меньше всего предположений. Можно начать с нее.


Если выполнены условия центральной предельной теоремы, то средние значения выборок из распределения распределены нормально.
Поэтому средний чек на пользователя можно моделировать нормальным распределением.

## Генерация данных

### Центральная предельная теорема.

Например, для распределения Парето:

In [ ]:
# pareto pdf = b / x^(b+1), b>0, x>=1
# lomax pdf = c / (x+1)^c, c>0, x>=1 (pareto with loc=-1)
lomax_x = np.linspace(0, 10, 100)

fig = go.Figure()
fig.add_trace(go.Scatter(x=lomax_x, y=stats.pareto.pdf(lomax_x, 3.5, loc=-1), name='pareto from 0, a=3.5'))
fig.add_trace(go.Scatter(x=lomax_x, y=stats.pareto.pdf(lomax_x, 1.5, loc=-1), name='pareto from 0, a=1.5'))
fig.show()

In [ ]:
n_samples = 3000
n_points_in_sample = 10000

results = np.random.pareto(1.5, size=[n_samples, n_points_in_sample])

means = np.array(list(map(np.mean, results)))
#means

fig = go.Figure()
fig.add_trace(go.Histogram(x=means, nbinsx=1000))
#fig.add_trace(go.Histogram(x=means[means<10], nbinsx=1000))
fig.show()

In [ ]:
n_samples = 3000
n_points_in_sample = 10000

results = np.random.pareto(3.5, size=[n_samples, n_points_in_sample])

means = np.array(list(map(np.mean, results)))
#means

fig = go.Figure()
fig.add_trace(go.Histogram(x=means, nbinsx=100))
#fig.add_trace(go.Histogram(x=means[means<10], nbinsx=1000))
fig.show()

У распределения Парето с параметром $1 \lt a \le 2$ дисперсия не конечная.
Центральная предельная теорема как раз требует от распределения 
определенного значения среднего и конечного значения дисперсии.

Но вроде есть обобщение центральной предельной теоремы на случай 
медленно убывающих распределений типа Парето.


Модель
Пришло N пользователей.
Каждый совершил от 0 до (бесконечности) покупок; 
предположим, что покупки независимы (что вряд ли) и вероятность каждой покупки p.
Чек каждой покупки - 3, 5, 10, 20$.

Количество покупок одним пользователем задается биномиальным распределением с числом попыток $n \to \infty$.  
Вероятность покупки $p$ также снижается.  
Это распределение Пуассона (https://en.wikipedia.org/wiki/Binomial_distribution#Poisson_approximation).  
Чек каждой покупки выбирается случайно.  

In [ ]:
n = 3000
n_pur_users = np.random.poisson(lam=1.5, size=n)
checks = [3, 5, 10, 20]

pur_sums = np.array(list(map(lambda npur: np.sum(np.random.choice(checks, npur, replace=True)), n_pur_users)))

fig = go.Figure()
fig.add_trace(go.Histogram(x=pur_sums, nbinsx=100))
#fig.add_trace(go.Histogram(x=means[means<10], nbinsx=1000))
fig.show()

In [ ]:
n_trials = 1000
n_users = 3000
n_pur_users = np.random.poisson(lam=1.5, size=[n_trials, n_users])

checks = [3, 5, 10, 20]

pur_sums = []
for trial in n_pur_users:
    pur_sums.append(np.array([np.sum(np.random.choice(checks, npur, replace=True)) for npur in trial]))
#pur_sums

mean_sums = [np.mean(x) for x in pur_sums]
#mean_sums

fig = go.Figure()
fig.add_trace(go.Histogram(x=mean_sums, nbinsx=50))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=np.concatenate(pur_sums, axis=0), nbinsx=100))
#fig.add_trace(go.Histogram(x=means[means<10], nbinsx=1000))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=mean_sums, nbinsx=100))
fig.show()

### Моделирование среднего чека по одному среднему значению

Есть выборка пользователей с покупками.


In [ ]:
n = 1000000
checks = [3, 5, 10, 20]

n_pur_users_a = np.random.poisson(lam=1.5, size=n)
pur_sums_a = np.array(list(map(lambda npur: np.sum(np.random.choice(checks, npur, replace=True)), n_pur_users_a)))

n_pur_users_b = np.random.poisson(lam=1.7, size=n)
pur_sums_b = np.array(list(map(lambda npur: np.sum(np.random.choice(checks, npur, replace=True)), n_pur_users_b)))



In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=pur_sums_a, nbinsx=100, name='A'))
fig.add_trace(go.Histogram(x=pur_sums_b, nbinsx=100, name='B'))
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.5)
#fig.add_trace(go.Histogram(x=means[means<10], nbinsx=1000))
fig.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Box(x=pur_sums_a, name='A'))
# fig.add_trace(go.Box(x=pur_sums_b, name='B'))
# fig.show()

In [ ]:
mean_a = np.mean(pur_sums_a)
mean_b = np.mean(pur_sums_b)
print(mean_a, mean_b)

print(np.std(pur_sums_a) / np.sqrt(len(pur_sums_a)))

Нормальное распределение задается двумя параметрами - среднее и стандартное отклонение.
Можно задать сетку из двух параметров.

In [ ]:
grid_points = 1001
mean_grid = np.linspace(10, 20, grid_points) #todo: use mean_a
mean_grid_prior = [ 1.0 / grid_points for x in mean_grid]
grid_points = 301
sigma_grid = np.linspace(0.001, 0.05, grid_points)
sigma_grid_prior = [ 1.0 / grid_points for x in sigma_grid]

grid = pd.merge(pd.DataFrame({'mean': mean_grid, 'mean_prior': mean_grid_prior, 'tmp_merge_key': 0}),
                pd.DataFrame({'sigma': sigma_grid, 'sigma_prior': sigma_grid_prior, 'tmp_merge_key': 0}),
                how='outer', 
                on='tmp_merge_key')
grid = grid[['mean', 'sigma', 'mean_prior', 'sigma_prior']]
grid.head()

In [ ]:
grid['A(data|mean, sigma)'] = grid.apply(
    lambda row: stats.norm.pdf(mean_a, loc=row['mean'], scale=row['sigma']),
    axis = 1)
grid['A_posterior'] = grid['A(data|mean, sigma)'] * grid['mean_prior'] * grid['sigma_prior']
grid['A_posterior'] = grid['A_posterior'] / sum(grid['A_posterior'])

grid['B(data|mean, sigma)'] = grid.apply(
    lambda row: stats.norm.pdf(mean_b, loc=row['mean'], scale=row['sigma']),
    axis = 1)
grid['B_posterior'] = grid['B(data|mean, sigma)'] * grid['mean_prior'] * grid['sigma_prior']
grid['B_posterior'] = grid['B_posterior'] / sum(grid['B_posterior'])
grid.head()

In [ ]:
# todo: avoid overlapping scales
fig = go.Figure()
fig.add_trace(go.Heatmap(z=grid['A_posterior'],
                         x=grid['mean'],
                         y=grid['sigma'],
                         opacity=0.5))
fig.add_trace(go.Heatmap(z=grid['B_posterior'],
                         x=grid['mean'],
                         y=grid['sigma'],
                         opacity=0.5))
fig.update_layout(title='Posterior Probability Density',
                  xaxis_title='Mean',
                  yaxis_title='Sigma')
fig.show()

### Бутстрап

Кажется, что строить модель по одной точке не очень надежно.  
Но других точек нет.  

Можно попробовать использовать бутстрап, чтобы нагенерировать еще точек.  
Непонятно, к чему это может привести, и как это скажется на точности.  
Но кажется, что катастрофических изменений быть не должно.

In [ ]:
#jackknife_means

jk_means_a = [(mean_a * len(pur_sums_a) - x) / (len(pur_sums_a) - 1)  for x in pur_sums_a]
jk_means_a

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=jk_means_a, nbinsx=100))
fig.show()

In [ ]:
#bootstrap means
bs_trials = 10000 

bs_means_a = [np.mean(np.random.choice(pur_sums_a, len(pur_sums_a), replace=True)) for i in range(bs_trials)]
bs_means_a

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=bs_means_a, nbinsx=100))
fig.show()

In [ ]:
# grid['A logprob(data|mean, sigma)'] = grid.apply(
#     lambda row: np.sum(stats.norm.logpdf(bs_means_a, loc=row['mean'], scale=row['sigma'])),
#     axis = 1)
# grid['A_posterior_log'] = grid['A logprob(data|mean, sigma)'] + np.log(grid['mean_prior'] * grid['sigma_prior'])
# grid['A_posterior_log_norm'] = grid['A_posterior_log'] - np.sum(grid['A_posterior_log'])
# grid['A_posterior'] = grid['A_posterior_log_norm'].apply('exp') 
# grid.head()

grid['A prob(data|mean, sigma)'] = grid.apply(
     lambda row: np.prod(stats.norm.pdf(bs_means_a, loc=row['mean'], scale=row['sigma'])),
     axis = 1)
grid['A_posterior'] = grid['A prob(data|mean, sigma)'] * grid['mean_prior'] * grid['sigma_prior']
grid['A_posterior_norm'] = grid['A_posterior'] / np.sum(grid['A_posterior'])
grid.head()

In [ ]:
# todo: avoid overlapping scales
fig = go.Figure()
fig.add_trace(go.Heatmap(z=grid['A_posterior'],
                         x=grid['mean'],
                         y=grid['sigma'],
                         opacity=0.5))
fig.add_trace(go.Heatmap(z=grid['B_posterior'],
                         x=grid['mean'],
                         y=grid['sigma'],
                         opacity=0.5))
fig.update_layout(title='Posterior Probability Density',
                  xaxis_title='Mean',
                  yaxis_title='Sigma')
fig.show()

Известно, что средние в пределе описываются нормальным распределением с параметрами 
$(\mu, \sigma^2/\sqrt{N})$. Есть ощущение, что вместо бутстрапа можно было бы насэмлить точек из этого распределения.

### Аналитическое выражение

https://en.wikipedia.org/wiki/Conjugate_prior

В качестве сопряженного априорного распределения к нормальному можно выбрать  
https://en.wikipedia.org/wiki/Normal-gamma_distribution

(если варьировать и $\mu$, и $\sigma$).

Если только $\mu$, будет проще - сопряженным априорным распределением также будет нормальное.

См. также https://www.cs.ubc.ca/~murphyk/Papers/bayesGauss.pdf 

Про использование NG-модели - см. https://stackoverflow.com/a/53367519

In [ ]:
### Сравнение двух нормальных распределений

In [ ]:
mean_a = 1.5
sigma_a = 0.7
mean_b = 5.0
sigma_b = 1.0

n_samples = 300
sample_a = np.random.normal(loc=mean_a, scale=sigma_a, size=n_samples)
sample_b = np.random.normal(loc=mean_b, scale=sigma_b, size=n_samples)


print('sample_a: ', sample_a[:3], '...')
print('exact mean_a: {}, exact sigma_a: {}'.format(mean_a, sigma_a))
print('exact mean_b: {}, exact sigma_b: {}'.format(mean_b, sigma_b))
display(pd.concat([
    pd.Series(sample_a).describe().rename('sample A').to_frame().T,
    pd.Series(sample_b).describe().rename('sample B').to_frame().T]))



x = np.linspace(start=-10, stop=10, num=200)
ya = stats.norm.pdf(x, loc=mean_a, scale=sigma_a)
yb = stats.norm.pdf(x, loc=mean_b, scale=sigma_b)

fig = go.Figure()
fig.add_trace(go.Histogram(x=sample_a, histnorm='probability density', 
                           name='Samples A', marker_color='red',
                           opacity=0.6))
fig.add_trace(go.Histogram(x=sample_b, histnorm='probability density', 
                           name='Samples B', marker_color='blue',
                           opacity=0.6))
fig.add_trace(go.Scatter(x=x, y=ya, name='A', line_color='red'))
fig.add_trace(go.Scatter(x=x, y=yb, name='B', line_color='blue'))
fig.update_layout(title='Normal Distributions',
                  xaxis_title='',
                  yaxis_title='Prob Density',
                  barmode='overlay')
fig.show()



In [ ]:
grid_points = 1001
mean_grid = np.linspace(1, 10, grid_points)
mean_grid_prior = [ 1.0 / grid_points for x in mean_grid]
grid_points = 101
sigma_grid = np.linspace(0.1, 3, grid_points)
sigma_grid_prior = [ 1.0 / grid_points for x in sigma_grid]

grid = pd.merge(pd.DataFrame({'mean': mean_grid, 'mean_prior': mean_grid_prior, 'tmp_merge_key': 0}),
                pd.DataFrame({'sigma': sigma_grid, 'sigma_prior': sigma_grid_prior, 'tmp_merge_key': 0}),
                how='outer', 
                on='tmp_merge_key')
grid = grid[['mean', 'sigma', 'mean_prior', 'sigma_prior']]
grid.head()

In [ ]:
grid['A logprob(data|mean, sigma)'] = grid.apply(
    lambda row: np.sum(stats.norm.logpdf(sample_a, loc=row['mean'], scale=row['sigma'])),
    axis = 1)
grid['A prob(data|mean, sigma)_fromlog'] = grid['A logprob(data|mean, sigma)'].apply('exp')
grid['A_posterior'] = grid['A prob(data|mean, sigma)_fromlog'] * grid['mean_prior'] * grid['sigma_prior']
grid['A_posterior'] = grid['A_posterior'] / sum(grid['A_posterior'])
grid.head()

In [ ]:
grid['B logprob(data|mean, sigma)'] = grid.apply(
    lambda row: np.sum(stats.norm.logpdf(sample_b, loc=row['mean'], scale=row['sigma'])),
    axis = 1)
grid['B prob(data|mean, sigma)_fromlog'] = grid['B logprob(data|mean, sigma)'].apply('exp')
grid['B_posterior'] = grid['B prob(data|mean, sigma)_fromlog'] * grid['mean_prior'] * grid['sigma_prior']
grid['B_posterior'] = grid['B_posterior'] / sum(grid['B_posterior'])
grid.head()

In [ ]:
# display(max(grid['A prob(data|mean, sigma)_fromlog']))
# grid[grid['A_posterior'] > 0.001 ].head()

In [ ]:
# todo: avoid overlapping scales
fig = go.Figure()
fig.add_trace(go.Heatmap(z=grid['A_posterior'],
                         x=grid['mean'],
                         y=grid['sigma'],
                         opacity=0.5))
fig.add_trace(go.Heatmap(z=grid['B_posterior'],
                         x=grid['mean'],
                         y=grid['sigma'],
                         opacity=0.5))
fig.update_layout(title='Posterior Probability Density',
                  xaxis_title='Mean',
                  yaxis_title='Sigma')
fig.show()

In [ ]:
A_mean_integrated_by_sigma = grid.groupby('mean').agg(lambda z: np.trapz(z['A_posterior'], x=z['sigma'])).iloc[:,0]
A_mean_integrated_by_sigma = A_mean_integrated_by_sigma.rename('A_post_integr_over_sigma').reset_index()
display(A_mean_integrated_by_sigma.loc[A_mean_integrated_by_sigma['A_post_integr_over_sigma'].idxmax()])

B_mean_integrated_by_sigma = grid.groupby('mean').agg(lambda z: np.trapz(z['B_posterior'], x=z['sigma'])).iloc[:,0]
B_mean_integrated_by_sigma = B_mean_integrated_by_sigma.rename('B_post_integr_over_sigma').reset_index()
display(B_mean_integrated_by_sigma.loc[B_mean_integrated_by_sigma['B_post_integr_over_sigma'].idxmax()])


fig = go.Figure()
fig.add_trace(go.Scatter(x=A_mean_integrated_by_sigma['mean'], 
                         y=A_mean_integrated_by_sigma['A_post_integr_over_sigma'],
                         name='Mean A',
                         mode='lines+markers'))
fig.add_trace(go.Scatter(x=B_mean_integrated_by_sigma['mean'], 
                         y=B_mean_integrated_by_sigma['B_post_integr_over_sigma'],
                         name='Mean B',
                         mode='lines+markers'))
fig.update_layout(title='Posterior',
                  xaxis_title='mean',
                  yaxis_title='Prob',
                  hovermode="x")
fig.show()


In [ ]:
### Оценка параметров распределения

Из сетки с вероятностями параметров, можно получить оценку на наиболее вероятные значения параметров.
Типа, 91-процентный интервал (HPDI) для mu и sigma.

Это здорово и увлекательно. Но не то, что нужно.
Интересуют не параметры модели, а какое распределение будет давать "большие" результаты.

In [ ]:
### Сравнение распределений

Первое - точечные оценки.
Т.е. сравнение средней величины и дисперсии в каждом.
Считается среднее. Либо аналитически, либо сэмплируется.





In [ ]:
Далее смотрится P(A) > P (B) .
Это эквивалентно Z = A - B. P(Z) > 0. 

Т.е. определяется новая случайная величина. Смотрится ее распределение.
Она показывает насколько A больше B и с какой вероятностью.
Особое внимание на накопленную сумму P(A-B > 0).

То же для A / B. см. https://en.wikipedia.org/wiki/Ratio_distribution
То же самое. В итоге распределение P(A/B). 
Обратить внимание на накопленную сумму вероятности P(A/B) > 1. 
По идее должна совпасть с P(A - B) > 0.
С распределением a/b особенность в том, что не определено среднее.


In [ ]:
n_sample = 100000
posterior_a = grid['A_posterior']
posterior_b = grid['B_posterior']
grid_idx = range(0, posterior_a.size)
pars_post_sample_a = np.random.choice(grid_idx, size=n_sample, p=posterior_a)
pars_post_sample_b = np.random.choice(grid_idx, size=n_sample, p=posterior_b)
#use np.map
sample_a = [np.random.normal(loc=grid['mean'][i], scale=grid['sigma'][i]) for i in pars_post_sample_a]
sample_b = [np.random.normal(loc=grid['mean'][i], scale=grid['sigma'][i]) for i in pars_post_sample_b]
z = [b - a for a,b in zip (sample_a, sample_b)]
#z
c = [b / a for a,b in zip (sample_a, sample_b)]

In [ ]:
zz = np.array(z)
diff = np.linspace(start=0, stop=10, num=100)
accum_prob = [len(zz[zz >= x]) / len(zz) for x in diff]

print("B-A > 0: ", len(zz[zz >= 0]) / len(zz))

fig = go.Figure()
fig.add_trace(go.Histogram(x=z, histnorm='probability density', 
                           name='B-A', marker_color='red',
                           opacity=0.6))
fig.add_trace(go.Scatter(x=diff, y=accum_prob, name='B-A > x', line_color='red'))

fig.update_layout(title='B-A',
                  xaxis_title='B-A',
                  yaxis_title='Prob Density',
                  barmode='overlay')
fig.show()


In [ ]:
cc = np.array(c)
frac = np.linspace(start=0.1, stop=10, num=100)
accum_prob = [len(cc[cc >= x]) / len(c) for x in frac]

print("B/A > 1: ", len(cc[cc >= 1]) / len(cc))

fig = go.Figure()
fig.add_trace(go.Histogram(x=cc, histnorm='probability density', 
                           name='A/B', marker_color='red',
                           xbins=dict(start=-10.0, end=10.0, size=0.3),
                           opacity=0.6))
fig.add_trace(go.Scatter(x=frac, y=accum_prob, name='B/A > x', line_color='red'))

fig.update_layout(title='B/A',
                  xaxis_title='B/A',
                  yaxis_title='Prob Density',
                  barmode='overlay')
fig.show()


In [ ]:
### Зачем нужна модель?
Почему бы просто не посчитать распределение разности с помощью ресемплинга?

См. permutation test и 
См. https://arxiv.org/abs/1411.5279 ?

In [ ]:
Сравнить с исходными выборками.

In [ ]:
n_rs = 100000
diff_rs = np.random.choice(sample_b, n_rs) - np.random.choice(sample_a, n_rs)
diff_grid_rs = np.linspace(start=0, stop=10, num=100)
accum_prob_rs = [len(diff_rs[diff_rs >= x]) / len(diff_rs) for x in diff_grid_rs]

print("rs B-A > 0: ", len(diff_rs[diff_rs >= 0]) / len(diff_rs))
print("B-A > 0: ", len(zz[zz >= 0]) / len(zz))


fig = go.Figure()
fig.add_trace(go.Histogram(x=diff_rs, histnorm='probability density', 
                           name='B-A', marker_color='red',
                           opacity=0.6))
fig.add_trace(go.Scatter(x=diff_grid_rs, y=accum_prob_rs, name='B-A > x', line_color='red'))

fig.add_trace(go.Histogram(x=z, histnorm='probability density', 
                           name='B-A', marker_color='blue',
                           opacity=0.6))
fig.add_trace(go.Scatter(x=diff, y=accum_prob, name='B-A > x', line_color='blue'))


fig.update_layout(title='B-A',
                  xaxis_title='B-A',
                  yaxis_title='Prob Density',
                  barmode='overlay')
fig.show()

In [ ]:
n_rs = 100000
rat_rs = np.random.choice(sample_b, n_rs) / np.random.choice(sample_a, n_rs)
rat_grid_rs = np.linspace(start=0.1, stop=10, num=100)
rat_accum_prob_rs = [len(rat_rs[rat_rs >= x]) / len(rat_rs) for x in rat_grid_rs]

print("rs B/A > 1: ", len(rat_rs[rat_rs >= 0]) / len(rat_rs))
print("B/A > 1: ", len(cc[cc >= 1]) / len(cc))


fig = go.Figure()
fig.add_trace(go.Histogram(x=rat_rs, histnorm='probability density', 
                           name='B/A', marker_color='red',
                           xbins=dict(start=-10.0, end=10.0, size=0.3),
                           opacity=0.6))
fig.add_trace(go.Scatter(x=rat_grid_rs, y=rat_accum_prob_rs, name='B/A > 1', line_color='red'))

fig.add_trace(go.Histogram(x=cc, histnorm='probability density', 
                           name='A/B', marker_color='blue',
                           xbins=dict(start=-10.0, end=10.0, size=0.3),
                           opacity=0.6))
fig.add_trace(go.Scatter(x=frac, y=accum_prob, name='B/A > x', line_color='blue'))


fig.update_layout(title='B/A',
                  xaxis_title='B/A',
                  yaxis_title='Prob Density',
                  barmode='overlay')
fig.show()

In [ ]:
На глаз не отличишь.
Тогда зачем все-таки нужна модель?

С моделью проще что-то делать аналитически ...

In [ ]:
### Зависимость от размера выборки

Исходный пример - много точек и заведомо видимое различие.

Отдельно - построить зависимость P(Z > 0) в зависимости от размера выборки. 

In [ ]:
mean_a = 1.5
sigma_a = 0.7
mean_b = mean_a * 1.3
sigma_b = sigma_a * 1.5

n_rs = 100000
diff_grid_rs = np.linspace(start=0, stop=10, num=100)

fig = go.Figure()

for n_samples, col in zip([30, 100, 300, 1000, 3000, 10000], ["red", "green", "blue", "orange", "cyan", "black"]):
    sample_a_diffn = np.random.normal(loc=mean_a, scale=sigma_a, size=n_samples)
    sample_b_diffn = np.random.normal(loc=mean_b, scale=sigma_b, size=n_samples)

    diff_rs = np.random.choice(sample_b_diffn, n_rs) - np.random.choice(sample_a_diffn, n_rs)
    accum_prob_rs = [len(diff_rs[diff_rs >= x]) / len(diff_rs) for x in diff_grid_rs]

    print("rs B-A > 0: ", len(diff_rs[diff_rs >= 0]) / len(diff_rs))

    fig.add_trace(go.Histogram(x=diff_rs, histnorm='probability density', 
                           name=str(n_samples), marker_color=col,
                           opacity=0.3))
    fig.add_trace(go.Scatter(x=diff_grid_rs, y=accum_prob_rs, name=str(n_samples), line_color=col))


fig.update_layout(title='B-A',
                  xaxis_title='B-A',
                  yaxis_title='Prob Density',
                  barmode='overlay')
fig.show()

In [ ]:
### Как выбрать один из вариантов 

Можно ли быть уверенным, что результат получился не случайно?

Если повторить эксперимент несколько раз, как будет меняться оценка?

In [ ]:
P(x>y | data) = P(data | x>y) * P(x>y) / P(data) ?

P(A-B > x | data) = P(data | A-B > x) * P(A-B > x) / P(data)

In [ ]:
### Длительность

In [ ]:
Предположим, что угадали параметры распределения в каждой из групп.  
Можно семлпировать значения из него.  
(вернее, семплировать все постериорное распределение).  
До тех пор, пока неопределенность не снизится до нужного уровня.  
Зная приток трафика, можно пересчитать это значение в количество дней. 

In [ ]:
# Пример на искусственных данных


In [ ]:
Смоделировать чеки и LTV с помощью распределения Паретто или BuyTillYouDie. 

Потом - понять как оценивать.

In [ ]:
# pareto pdf = b / x^(b+1), b>0, x>=1
# lomax pdf = c / (x+1)^c, c>0, x>=1 (pareto with loc=-1)
b = 1.16
pareto_x = np.linspace(1, 5, 100)
lomax_x = np.linspace(0, 5, 100)

fig = go.Figure()
fig.add_trace(go.Scatter(x=pareto_x, y=stats.pareto.pdf(pareto_x, b), name='pareto pdf'))
fig.add_trace(go.Scatter(x=lomax_x, y=stats.lomax.pdf(lomax_x, b), name='lomax'))
fig.add_trace(go.Scatter(x=lomax_x, y=stats.pareto.pdf(lomax_x, b, loc=-1), name='pareto loc'))
fig.show()

### Моделирование отдельных покупок

Buy Till You Die - Класс моделей для моделирования отдельных покупок.

https://cran.r-project.org/web/packages/BTYD/vignettes/BTYD-walkthrough.pdf?source=post_page

https://en.wikipedia.org/wiki/Buy_Till_you_Die